In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkb
import sqlalchemy
import re

## settings

In [ ]:
local_crs = 3006
osm_crs = 4326

row_limit_lines = 1000000
city_abbr = 'gbg'

write_to_table = True
plot = False

## read map-matched trajectories

In [ ]:
# read from database
url = sqlalchemy.URL.create(
    "postgresql+psycopg", port=5432,
    host="host", database="database", username="username")
engine = sqlalchemy.create_engine(url)

In [ ]:
n = 2 # n of samples to concatenate
i = 1 # sample to load (in case n=1)

if n == 1:
    sql_query_lines = """
        SELECT *
        FROM trajectories.{0}_trajectory_lines_2024_lcss_random{1}
        LIMIT {2};
        """.format(city_abbr, i, row_limit_lines)

    with engine.connect() as conn_flowsense:
        with conn_flowsense.execute(sqlalchemy.text(sql_query_lines)) as cursor:
            df_lines = pd.read_sql(sql_query_lines, con=conn_flowsense)

    mapmatched = gpd.GeoDataFrame(df_lines, geometry=df_lines['geometry'].apply(wkb.loads), crs=local_crs)

else:
    mapmatched = gpd.GeoDataFrame()

    for i in range(n):

        sql_query_lines = """
            SELECT *
            FROM trajectories.{0}_trajectory_lines_2024_lcss_random{1}
            LIMIT {2};
            """.format(city_abbr, i+1, row_limit_lines)

        with engine.connect() as conn_flowsense:
            with conn_flowsense.execute(sqlalchemy.text(sql_query_lines)) as cursor:
                df_lines = pd.read_sql(sql_query_lines, con=conn_flowsense)

        gdf = gpd.GeoDataFrame(df_lines, geometry=df_lines['geometry'].apply(wkb.loads), crs=local_crs)
        mapmatched = pd.concat([mapmatched, gdf], ignore_index=True)
        mapmatched = mapmatched.drop_duplicates(subset='traj_id', keep='first')

print(len(mapmatched))

In [ ]:
def road_ids_to_dict(roads_ids):
    pattern = r'RoadId\(start=(\d+), end=(\d+), key=(\d+)\)'
    matches = re.findall(pattern, roads_ids)
    return [{'start': int(start), 'end': int(end), 'key': int(key)} for start, end, key in matches]

In [ ]:
mapmatched['road_ids'] = mapmatched.road_ids.apply(lambda x: road_ids_to_dict(x))

In [ ]:
# split mapmatched trajectories per road segment
mapmatched_exploded = mapmatched.explode('road_ids').rename(columns={'road_ids':'road_id'}).reset_index()
mapmatched_exploded.road_id = mapmatched_exploded.road_id.apply(lambda x: (x['start'], x['end'], x['key']))

In [ ]:
len(mapmatched_exploded)

## read road network

In [ ]:
sql_query_edges = """
    SELECT *
    FROM road_network.trafikverket_edges_{};
    """.format(city_abbr)

with engine.connect() as conn_flowsense:
    with conn_flowsense.execute(sqlalchemy.text(sql_query_lines)) as cursor:
        df_edges = pd.read_sql(sql_query_edges, con=conn_flowsense)

edges = gpd.GeoDataFrame(df_edges, geometry=df_edges['geometry'].apply(wkb.loads), crs=local_crs)

In [ ]:
edges.set_index(['u', 'v', 'key'], inplace=True)
edges['road_id'] = edges.index

## From trajectories to flows for several trajectory subsets

In [ ]:
flows = edges[['osmid', 'RLID', 'maxspeed', 'geometry', 'road_id']].copy()

In [ ]:
avg_speed_thresholds = [0, 2.5, 5, 7.5, 10, 12.5, 15, 17.5, 20] # 0 means including all trajectories
npoints_thresholds = [2, 3, 4, 5, 6, 7, 8, 9, 10] # 2 means including all trajectories
avg_dist_thresholds = [1000, 900, 800, 700, 600, 500, 400, 300, 200, 100] # 1000 means including all trajectories

samplesize = 100000

In [ ]:
for threshold in avg_speed_thresholds:
    mapmatched_subset = (mapmatched_exploded[mapmatched_exploded.avg_speed >= threshold]).sample(n=samplesize, random_state=1)
    print(len(mapmatched_subset))

    col_trajcount = 'trajcount_minavgspeed{}'.format(threshold)
    col_relflow = 'relflow_minavgspeed{}'.format(threshold)

    # count number of unique trajectories per road segment
    edges_traj_ids = edges.merge(mapmatched_subset[['road_id', 'traj_id']], on='road_id', how='left')
    trajs_per_road = edges_traj_ids.groupby('road_id')['traj_id'].nunique()

    # flows: road network with trajectory counts
    flows = flows.merge(trajs_per_road, on='road_id', how='left')
    flows.rename(columns={'traj_id':col_trajcount}, inplace=True)

    flows[col_relflow] = flows[col_trajcount] / flows[col_trajcount].max()

In [ ]:
for threshold in npoints_thresholds:
    mapmatched_subset = (mapmatched_exploded[mapmatched_exploded.n_points >= threshold]).sample(n=samplesize, random_state=1)
    print(len(mapmatched_subset))

    col_trajcount = 'trajcount_minnpoints{}'.format(threshold)
    col_relflow = 'relflow_minnpoints{}'.format(threshold)

    # count number of unique trajectories per road segment
    edges_traj_ids = edges.merge(mapmatched_subset[['road_id', 'traj_id']], on='road_id', how='left')
    trajs_per_road = edges_traj_ids.groupby('road_id')['traj_id'].nunique()

    # flows: road network with trajectory counts
    flows = flows.merge(trajs_per_road, on='road_id', how='left')
    flows.rename(columns={'traj_id':col_trajcount}, inplace=True)

    flows[col_relflow] = flows[col_trajcount] / flows[col_trajcount].max()

In [ ]:
for threshold in avg_dist_thresholds:
    mapmatched_subset = (mapmatched_exploded[mapmatched_exploded.avg_dist <= threshold]).sample(n=samplesize, random_state=1)
    print(len(mapmatched_subset))

    col_trajcount = 'trajcount_maxavgdist{}'.format(threshold)
    col_relflow = 'relflow_maxavgdist{}'.format(threshold)

    # count number of unique trajectories per road segment
    edges_traj_ids = edges.merge(mapmatched_subset[['road_id', 'traj_id']], on='road_id', how='left')
    trajs_per_road = edges_traj_ids.groupby('road_id')['traj_id'].nunique()

    # flows: road network with trajectory counts
    flows = flows.merge(trajs_per_road, on='road_id', how='left')
    flows.rename(columns={'traj_id':col_trajcount}, inplace=True)

    flows[col_relflow] = flows[col_trajcount] / flows[col_trajcount].max()

In [ ]:
flows.drop(columns='road_id', inplace=True)

## write flows to table

In [ ]:
flows['geometry'] = flows.geometry.force_2d()

In [ ]:
flows.rename(columns={'osmid':'ISA_index'}, inplace=True)

In [ ]:
# write to database
if write_to_table:

    url_flowsense = sqlalchemy.URL.create(
        "postgresql+psycopg", port=5432,
        host="host", database="database", username="username")
    engine_flowsense = sqlalchemy.create_engine(url_flowsense)

    flows.to_postgis(
        name='{}_flows_2024_random1to{}'.format(city_abbr, n),
        con=engine_flowsense,
        schema='flows',
        if_exists='replace',
        index=False)